In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

import math
import matplotlib.colors as cx
import matplotlib.cm as cm

from glob import glob
from scipy import stats
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import MaxNLocator

%matplotlib inline

In [ ]:
path = r'G:\My Drive\Tuthill Lab Shared\Katie\thermal_experiments\data\snow_flies'
outpath = r'G:\My Drive\Tuthill Lab Shared\Katie\thermal_experiments\summaries'
outfile = os.path.join(outpath, 'snowfly_data.parquet')

In [ ]:
data = pd.DataFrame()
(root, dirs, files) = next(os.walk(path))

for session in dirs: 
    
    session_path = os.path.join(path, session)
    (r, flies, f) = next(os.walk(session_path))
    
    for fly in flies:
        
        fly_path = os.path.join(session_path, fly)
        (r, trials, f) = next(os.walk(fly_path))
        
        for trial in trials: 
            
            trial_path = os.path.join(fly_path, trial, 'temp_data_final.csv')
            video_path = glob(os.path.join(fly_path, trial, '*.csq'))
            if len(video_path) == 0:
                continue
                
            # print(video_path)
            video_name = os.path.basename(video_path[0])
            if not os.path.exists(trial_path):
                continue
                
            print(trial_path)
             
            trial_num = re.findall(r'\d+', trial)[0]
            trial_data = pd.read_csv(trial_path)
            trial_data['date'] = session
            trial_data['fly'] = fly
            trial_data['trial'] = trial_num
            trial_data['flyid'] = fly + '_' + trial_num
            trial_data['filename'] = video_name
            trial_data['fullfile'] = os.path.join(session, fly, trial, video_name)
            data = data.append(trial_data)
            
# save data to parquet file
data.to_parquet(outfile + '.gzip', compression = 'gzip')

In [ ]:
data = pd.read_parquet(outfile + '.gzip' , engine='fastparquet')

In [ ]:
data.replace(0, np.nan, inplace=True)

In [ ]:
def get_video_length(nframes, thresh = None, fps = 30):

    frames = np.arange(0, nframes, 1)
    time_s = frames / fps
    time_m = frames / (fps*60)
    
    if thresh is not None:
        time_s = time_s[:thresh]
        time_m = time_m[:thresh]
         
    return time_s, time_m

In [ ]:
# for each fly, plot the average temperature, max temperature, and cold plate temperature 
sns.set()
sns.set_style('ticks')
flies = np.unique(data.flyid)
max_frames = 54000

for fly in flies:
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    
    fig = plt.figure(figsize = (10, 6))
    plt.title(fly)
    plt.xlabel('time (minutes)')
    plt.ylabel('temperature ($^{\circ}$C)')
    
    plt.plot(t_m, fly_data.max_temp[:len(t_m)], 'xkcd:light blue', linewidth = 0.5, label = 'max snowfly temp')
    plt.plot(t_m, fly_data.avg_temp[:len(t_m)], 'xkcd:slate blue', linewidth = 0.5, label = 'avg snowfly temp')
    plt.plot(t_m, fly_data.cold_plate_temp[:len(t_m)], 'xkcd:black', linewidth = 0.5, label = 'cold plate temp')
    
    sns.despine()
    plt.legend()
    plt.show()

In [ ]:
# for each fly, plot the x position and y position across time 
sns.set()
sns.set_style('ticks')
flies = np.unique(data.flyid)

for fly in flies:
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    
    fig = plt.figure(figsize = (10, 6))
    ax = plt.gca()
    plt.title(fly)
    plt.xlabel('time (minutes)')
    plt.ylabel('x position (pixels)')   
    plt.plot(t_m, fly_data.x[:len(t_m)], 'r', linewidth = 0.5, label = 'unfiltered')
    plt.plot(t_m, fly_data.x_filt[:len(t_m)], 'k', linewidth = 0.5, label = 'filtered')
    sns.despine()
    hs, ls = ax.get_legend_handles_labels()
    plt.show()
    
    fig = plt.figure(figsize = (10, 6))
    plt.title(fly)
    plt.xlabel('time (minutes)')
    plt.ylabel('y position (pixels)')   
    plt.plot(t_m, fly_data.y[:len(t_m)], 'r', linewidth = 0.5, label = 'unfiltered')
    plt.plot(t_m, fly_data.y_filt[:len(t_m)], 'k', linewidth = 0.5, label = 'filtered')
    sns.despine()
    plt.show()
    
fig = plt.figure(figsize = (3,3))
plt.legend(handles = hs, labels = ls)
plt.axis('off')
plt.show()
    
    

In [ ]:
# for each fly, plot the position of the fly in the metal ring
sns.set()
sns.set_style('ticks')
flies = np.unique(data.flyid)
max_frames = 54000
# last_frame = len(t_m)
last_frame = 36000
last_frame = 31505
fps = 30
seconds = 60

for fly in flies:
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    
    trial_path = os.path.dirname(fly_data.fullfile.iloc[0])
    # mask_path = os.path.join(path, trial_path, 'mask - Ellipse 1.bmp')
    # mask = plt.imread(mask_path)
    
    fig = plt.figure()
    ax = plt.gca()
    plt.title('Snow fly activity')
    # plt.title(fly)
    #plt.xlabel('x position (pixels)')
    #plt.ylabel('y position (pixels)')
    # ax.imshow(mask)
    xs = fly_data.x_filt[:last_frame]
    ys = fly_data.y_filt[:last_frame]
    ax.plot(ys, xs, 'k', linewidth = 0.5)
    
    start = 13000
    d = seconds*fps
    ax.plot(fly_data.y_filt[start:start+d], fly_data.x_filt[start:start+d], 'xkcd:red', linewidth = 1)
    ax.axis('equal')
    
    sns.despine()
    ax.invert_yaxis()
    plt.axis('off')
    plt.show()

In [ ]:
# difference between the average fly temperature and the cold plate temperature 
sns.set()
sns.set_style('ticks')
flies = np.unique(data.flyid)

for fly in flies:
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    delta_temp = fly_data.avg_temp - fly_data.cold_plate_temp
    
    fig = plt.figure(figsize = (10, 6))
    ax = plt.gca()
    #ax.spines['left'].set_position('zero')
    #ax.spines['bottom'].set_position('zero')
    plt.title(fly)
    plt.xlabel('time (minutes)')
    plt.ylabel('movement')  
    
    plt.plot(t_m, delta_temp[:len(t_m)], 'k', linewidth = 0.5)
    plt.axhline(y = 0, color = 'k', linewidth = 1, linestyle = ':') 
    
    sns.despine()
    hs, ls = ax.get_legend_handles_labels()
    plt.show()


In [ ]:
# cold plate temperature for all trials - how consistent is the ramp? 
sns.set()
sns.set_style('ticks')
flies = np.unique(data.flyid)
colors = sns.color_palette('mako', len(flies))

fig = plt.figure(figsize = (10, 6))
plt.title('cold plate temperatures')
plt.xlabel('time (minutes)')
plt.ylabel('temperature ($^{\circ}$C)')

for j, fly in enumerate(flies):
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    plt.plot(t_m, fly_data.cold_plate_temp[:len(t_m)], color = colors[j], linewidth = 0.5)
    
sns.despine()
plt.show()


In [ ]:
# snow fly temperatures for all trials
sns.set()
sns.set_style('ticks')
flies = np.unique(data.flyid)
colors = sns.color_palette('mako', len(flies))

fig = plt.figure(figsize = (10, 6))
ax = plt.gca()
plt.title('avg snowfly temperatures')
plt.xlabel('time (minutes)')
plt.ylabel('temperature ($^{\circ}C$)')

for j, fly in enumerate(flies):
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    plt.plot(t_m, fly_data.avg_temp[:len(t_m)], color = colors[j], linewidth = 0.5, label = fly)

hs, ls = ax.get_legend_handles_labels()
sns.despine()
plt.show()

fig = plt.figure(figsize = (3,3))
plt.axis('off')
plt.legend(handles = hs, labels = ls)
sns.despine()
plt.show()


In [ ]:
# plot the pixel intensity across time for each fly
sns.set()
sns.set_style('ticks')
fs = 14
flies = np.unique(data.flyid)
colors = sns.color_palette('mako', 3)

for j, fly in enumerate(flies):
    
    fig = plt.figure(figsize = (8, 5))
    ax = plt.gca()
    plt.title(fly, fontsize = fs)
    ax.set_xlabel('time (minutes)', fontsize = fs)
    ax.set_ylabel('temperature ($^{\circ}$C)', fontsize = fs)
    
    fly_data = data[data.flyid == fly]
    cp_temp = np.array(fly_data.cold_plate_temp[:len(t_m)])
    cp_temp[cp_temp == 0] = np.nan 
    fly_temp = np.array(fly_data.avg_temp[:len(t_m)])
    fly_temp[fly_temp == 0] = np.nan 
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)    
    ax.plot(t_m, cp_temp, color = colors[0], linewidth = 0.5, label = 'average cold plate temp')
    ax.plot(t_m, fly_temp, color = colors[1], linewidth = 0.5, label = 'average snow fly temp')

    ax2 = ax.twinx()
    ax2.set_ylabel('movement (norm.)', fontsize = fs)
    movement_norm = fly_data.movement[:len(t_m)] / max(fly_data.movement[:len(t_m)])
    ax2.plot(t_m, movement_norm, color = colors[2], linewidth = 0.5, label = 'activity', alpha= 0.8)
    
    hs, ls = ax.get_legend_handles_labels()
    sns.despine(right = False)
    plt.show()

fig = plt.figure(figsize = (3,3))
plt.axis('off')
plt.legend(handles = hs, labels = ls, fontsize = fs)
plt.show()


In [ ]:
path = r'G:\My Drive\Tuthill Lab Shared\Katie\thermal_experiments\data\cold_tolerance_summary - snow flies.csv'
data = pd.read_csv(path)
data = data[data['scp_germanium_temp'].isnull()]
exclude_dates = ['10.30.20', '11.6.20', '11.13.20', '11.20.20', '12.4.20', '12.11.20']
exclude_flies = ['SF0036', 'SF0075']
data = data[~data.date.isin(exclude_dates)]
data = data[~data.fly_id.isin(exclude_flies)]

In [ ]:
scp_ambient = data[~data['scp_ambient_temp'].isnull()]['scp_ambient_temp'].astype(float)
scp_fly_pre = data[~data['scp_fly_temp_pre'].isnull()]['scp_fly_temp_pre'].astype(float)
scp_fly_post = data[~data['scp_fly_temp_post'].isnull()]['scp_fly_temp_post'].astype(float)

cc_ambient = data[~data['cc_ambient_temp'].isnull()]['cc_ambient_temp']
cc_fly = data[~data['cc_fly_temp'].isnull()]['cc_fly_temp']
recovery_ambient = data[~data['recovery_ambient_temp'].isnull()]['recovery_ambient_temp']
recovery_fly = data[~data['recovery_fly_temp'].isnull()]['recovery_fly_temp']

In [ ]:
# # histograms of chill coma and chill coma recovery temperatures  
# sns.set()
# sns.set_style('ticks')

# figure = plt.figure(figsize = (9, 5))
# plt.title('Distribution of snow fly temperatures', fontsize = 14)
# plt.ylabel('Counts', fontsize = 14)
# plt.xlabel('Temperature ($^{\circ}$C)', fontsize = 14)
# plt.xlim([-15, 15])

# edges = np.arange(-15, 15, 1.5)
# #plt.hist(cc_ambient, bins = edges, color = colors[0], alpha = 0.8, label = 'cold plate temp')
# plt.hist(cc_fly, bins = edges, color = 'k', alpha = 0.8, label = 'Critical thermal minimum ($CT_{min}$)')
# #plt.hist(recovery_ambient, bins = edges, color = colors[2], alpha = 0.8, label = 'cold plate temp (recovery)')
# plt.hist(recovery_fly, bins = edges, color = '#999999', alpha = 0.8, label = 'Chill coma recovery temperature')

# plt.legend(fontsize = 13, loc = 'upper right')
# sns.despine()
# plt.show()

In [ ]:
# histograms of chill coma and supercooling 
sns.set()
sns.set_style('ticks')

figure = plt.figure(figsize = (8, 5))
plt.title('Distribution of snow fly temperatures', fontsize = 14)
plt.ylabel('Counts', fontsize = 14)
plt.xlabel('Temperature ($^{\circ}$C)', fontsize = 14)
plt.xlim([-14, 2])
plt.ylim([0, 20])

edges = np.arange(-14, 2, 1)
plt.hist(scp_fly_pre, bins = edges, color = 'r', alpha = 0.5, label = 'Supercooling point \n (n = ' + str(len(scp_fly_pre)) + ')')
plt.hist(cc_fly, bins = edges, color = 'b', alpha = 0.5, label = 'Critical thermal minimum \n (n = ' + str(len(cc_fly)) + ')')

ax = plt.gca()
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_yticks(ticks = np.arange(0, 21, 4))

plt.legend(fontsize = 13, loc = 'upper right')
sns.despine()
plt.show()

In [ ]:
# # histograms of supercooling points 
# sns.set()
# sns.set_style('ticks')
# colors = sns.color_palette('mako', 4)

# figure = plt.figure(figsize = (9, 5))
# plt.title('Distribution of supercooling points in snowflies', fontsize = 14)
# plt.ylabel('Counts', fontsize = 14)
# plt.xlabel('Temperature ($^{\circ}$C)', fontsize = 14)

# edges = np.arange(-20, 2, 1)
# scp_delta = abs(abs(scp_fly_pre) - abs(scp_fly_post))
# plt.hist(scp_ambient, bins = edges, color = 'k', alpha = 0.85, label = 'Cold plate temperature')
# plt.hist(scp_fly_pre, bins = edges, color = '#999999', alpha = 0.85, label = 'Snow fly temperature (pre-SCP)')
# plt.hist(scp_fly_post, bins = edges, color = 'w', edgecolor = 'k', alpha = 0.3, label = 'Snow fly temperature (post-SCP)')
# # plt.hist(scp_delta, bins = edges, color = colors[3], alpha = 0.85, label = 'Snow fly temp increase')

# plt.legend(fontsize = 13, loc = 'upper left')
# sns.despine()
# plt.show()